In [ ]:
#!pip uninstall tensorflow -y
#!pip install tf-nightly
!pip install tensorflow-model-optimization

     |████████████████████████████████| 174kB 8.0MB/s 
     |████████████████████████████████| 296kB 17.0MB/s 


In [ ]:
import json
import cv2 
import numpy as np
from PIL import Image, ImageFilter
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
import os

In [ ]:
tf.__version__

'2.2.0'

In [ ]:
with open('/content/drive/My Drive/Violette_art/via_export_json.json') as f:
  data = json.load(f)

# remove unlabelled data
delete = [key for key in data if data[key]['regions'] == []] 
# delete the key 
for key in delete: del data[key] 

In [ ]:
def get_mat(p1, p2):
  """
  Function to get the homography matrix that transforms points p1 to p2
  """
  A = []
  for i in range(0, len(p1)):
      x, y = p1[i][0], p1[i][1]
      u, v = p2[i][0], p2[i][1]
      A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
      A.append([0, 0, 0, x, y, 1, -v*x, -v*y, -v])
  A = np.asarray(A)
  U, S, Vh = np.linalg.svd(A)
  L = Vh[-1,:] / Vh[-1,-1]
  H = L.reshape(3, 3)
  return H

height = 480
width = 800
channels = 3
input_frame = np.zeros(shape=(len(data), 256, 256, 3))
output_frame = np.zeros(shape=(len(data), 8))
skip = []
h = int(height/32)
w = int(width/32)

#anchor point is front top left
A = np.array([[0,0], [25,0], [0, 100], [25, 100]])
for i, val in tqdm(enumerate(data)):
  try:
    rgb_image = Image.open("/content/drive/My Drive/Violette_art/all_frames/{}".format(data[val]['filename']))
    #First open, crop and resize the image
    area = (160, 60, 1760, 1020)
    cropped_img = rgb_image.crop(area)
    newsize = (800, 480) 
    resized_img = np.array(cropped_img.resize(newsize))
    #input_frame[i, :, :, :] = np.array(resized_img)
    min_x = 800
    max_x = 0
    min_y = 480
    max_y = 0
    B = {}
    break_bool = False
    for point in data[val]['regions']:
      label = point['region_attributes']['points']
      if label in ['ftr', 'ftl', 'btl', 'btr']:
        x, y = point['shape_attributes']['cx'], point['shape_attributes']['cy']
        if x > 160 and x < 1760:
          x = (x - 160) / 2
        else: 
          skip.append(i)
          break_bool = True
        if y > 60 and y < 1020:
          y = (y - 60) / 2
        else: 
          skip.append(i)
          break_bool = True
        B[label] = [x, y]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    if break_bool:
      continue
    
    min_y = int(min_y)
    max_y = int(max_y)
    min_x = int(min_x)
    max_x = int(max_x)
    #resize image again to 256X256
    img = Image.fromarray(resized_img.astype(np.uint8))
    img = img.resize((256, 256))
    input_frame[i] = np.array(img)
    #get the homography matrix
    B_sorted = np.array([B['ftl'], B['ftr'], B['btl'], B['btr']])
    M = get_mat(A, np.array(B_sorted))
    M = np.ndarray.flatten(M)
    #rescale the points to where they will be on the output image
    label = M[:-1]
    label[2] = B['ftl'][0] / 800
    label[5] = B['ftl'][1] / 480
    output_frame[i] = label
    
  except Exception as e:
    print(e)
    skip.append(i)

#delete the poorly formed data points
input_frame = np.delete(input_frame, skip, axis=0)
output_frame = np.delete(output_frame, skip, axis=0)
input_frame.shape, output_frame.shape

In [ ]:
#VGG style model that outputs the homography matrix
inputs = layers.Input(shape = (256, 256, 3))
x = layers.Conv2D(32, kernel_size=3, activation='relu')(inputs)
x = layers.Conv2D(32, kernel_size=3, activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(32, kernel_size=3, activation='relu')(x)
x = layers.Conv2D(32, kernel_size=3, activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu')(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu')(x)
x = layers.Conv2D(64, kernel_size=3, activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(1024)(x)
output = layers.Dense(8)(x)
model = keras.Model(inputs = inputs, outputs = outputs)
model.summary()

In [ ]:
end_step = np.ceil(1.0 * (.9 * input_frame.shape[0]) / 16).astype(np.int32)
end_step *= 100 #epochs
initial_sparsity=.5
final_sparsity=.95
begin_step=20
frequency=5

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                  final_sparsity=final_sparsity,
                                                  begin_step=begin_step,
                                                  end_step=end_step,
                                                  frequency=frequency)
}

model = sparsity.prune_low_magnitude(model, **new_pruning_params)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

optimizer = keras.optimizers.Adam()

def perturb(image, label):
  image = tf.image.random_brightness(image, max_delta=0.1) # Random brightness
  if np.random.randint(2, size=1)[0] == 0:
    image = tf.image.random_crop(image, size = [100, 100, 3])
    image = tf.image.resize(image, size=[128, 128])
  return image, label

x_train = ((input_frame / 128.0) - 1).astype(np.float32)

checkpoint_path = "/content/drive/My Drive/Violette_art/homography_small.tf"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
 
callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir='./logs'),
  sparsity.UpdatePruningStep(),
  cp_callback 
]

model.fit(x_train, output_frame, shuffle=True, batch_size=8, validation_split=.1, epochs=100, callbacks=callbacks)

In [ ]:
tflite_path = '/content/drive/My Drive/Violette_art/homography_small.tflite'
final_model = sparsity.strip_pruning(model)
tf.keras.models.save_model(final_model, checkpoint_path, include_optimizer=False)
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
tflite_quant_model = converter.convert()
with open(tflite_path, 'wb') as f:
  f.write(tflite_quant_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Violette_art/homography_small.tf/assets
